# PRÉ-AMBULO - IMPORT LIBS

In [ ]:
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image
import torchvision
import torch
import cv2
import os
import glob
import numpy as np

from modules.helper.helper import create_video_from_images, get_framerate

# V1 - Background Subtractor

In [ ]:
path        = "output\\*.jpg"
save        = "output\\sub_background"
os.makedirs(save, exist_ok=True)

files       = sorted(glob.glob(path), key=lambda x: int("".join(filter(str.isdigit, os.path.basename(x)))))

img_before  = None
background_stack = []  # Para armazenar frames sem movimento

for idx, filename in enumerate(files):
    frame = cv2.imread(filename)

    if img_before is not None:
        # Grayscale e equalização
        gray_prev = cv2.equalizeHist(cv2.cvtColor(img_before, cv2.COLOR_BGR2GRAY))
        gray_curr = cv2.equalizeHist(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        # Redução de ruído
        gray_prev = cv2.GaussianBlur(gray_prev, (5, 5), 0)
        gray_curr = cv2.GaussianBlur(gray_curr, (5, 5), 0)

        # Subtração e limiarização
        diff = cv2.absdiff(gray_prev, gray_curr)
        _, motion_mask = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

        # Remover pequenos ruídos
        motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

        # Inverter para pegar áreas "estáticas"
        static_mask = cv2.bitwise_not(motion_mask)

        # Aplicar máscara estática ao frame atual
        static_pixels = cv2.bitwise_and(frame, frame, mask=static_mask)
        background_stack.append(static_pixels)

        # Salvar máscara de movimento
        cv2.imwrite(os.path.join(save, f"results{idx}.jpg"), motion_mask)

    img_before = frame.copy()

# # Gerar background final pela mediana dos frames estáticos
# if background_stack:
#     background_array = np.stack(background_stack, axis=3)
#     background_final = np.median(background_array, axis=3).astype(np.uint8)
#     cv2.imwrite(os.path.join("background_final.jpg"), background_final)

# Exemplo de uso
image_folder = save
output_video_path = "Detect.mp4"
frame_rate = 13  # Substitua pelo frame rate desejado
create_video_from_images(image_folder, output_video_path, frame_rate)

# V2 - calcOpticalFlowFarneback

In [ ]:
# Caminho para o vídeo
video_path  = "input\\Teste.mp4"
save        = "output\\optical_flow"
os.makedirs(save, exist_ok=True)

# Captura de vídeo
cap = cv2.VideoCapture(video_path)

prev_gray = None
idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_gray is not None:
        
        # Calcula o fluxo óptico denso
        flow = cv2.calcOpticalFlowFarneback(
            prev_gray,      # imagem anterior (grayscale)
            gray,           # imagem atual (grayscale)
            None,           # matriz de fluxo (pode ser None)
            pyr_scale=0.25, # escala da pirâmide de imagem
            levels=3,       # número de níveis da pirâmide
            winsize=10,     # tamanho da janela
            iterations=5,   # número de iterações por nível
            poly_n=7,       # tamanho da vizinhança para polinômio
            poly_sigma=1.5, # sigma (desvio padrão) da Gaussiana usada
            flags=0         # opções adicionais (geralmente 0)
        )

        # Converte o fluxo em magnitude e ângulo
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Cria máscara binária para movimento baseado na magnitude
        movement_mask = cv2.threshold(mag, 1.5, 255, cv2.THRESH_BINARY)[1]
        movement_mask = movement_mask.astype(np.uint8)

        # Salva a máscara de movimento
        cv2.imwrite(os.path.join(save, f"results{idx}.jpg"), movement_mask)

        # # (Opcional) Mostra o vetor de movimento com cor HSV
        # hsv = np.zeros_like(frame)
        # hsv[..., 1] = 255
        # hsv[..., 0] = ang * 180 / np.pi / 2
        # hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        # flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        # cv2.imwrite(os.path.join(save, f"flow_visual_{idx:04}.jpg"), flow_rgb)

    prev_gray = gray
    idx+=1

# Exemplo de uso
image_folder = save
output_video_path = "motion_flow.mp4"
frame_rate = get_framerate(video_path)  # Substitua pelo frame rate desejado
create_video_from_images(image_folder, output_video_path, frame_rate)

# V3 - createBackgroundSubtractorKNN

In [ ]:
import cv2
import os
import numpy as np
import glob

# Caminhos
video_path = "input\\Teste.mp4"
save = "output\\knn_video"
os.makedirs(save, exist_ok=True)

# Inicializar captura e detector KNN
cap = cv2.VideoCapture(video_path)
fgbg = cv2.createBackgroundSubtractorKNN(history=30, dist2Threshold=400, detectShadows=False)

# Kernels para refinamento de máscaras
kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
kernel_close = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))

area = 100  # Área mínima para contornos (ajustar conforme necessário)

idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Ajuste de constraste
    frame_blur = cv2.GaussianBlur(frame, (3, 3), 0)
    frame = cv2.detailEnhance(frame_blur, sigma_s=30, sigma_r=0.1)

    # Ajuste automático de brilho
    # Converter para escala de cinza para análise de brilho
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Calcular o brilho médio
    mean_brightness = np.mean(gray_frame)
    # Definir brilho alvo (ajustar conforme necessário)
    target_brightness = 90
    # Calcular fator de ajuste
    adjustment_factor = target_brightness / mean_brightness if mean_brightness > 0 else 1
    # Aplicar ajuste de brilho
    frame = cv2.convertScaleAbs(frame, alpha=adjustment_factor, beta=0)

    # Redução de ruído
    frame_blur = cv2.GaussianBlur(frame, (7, 7), 0)
    
    # Grayscale e equalização
    frame_equalized = cv2.equalizeHist(cv2.cvtColor(frame_blur, cv2.COLOR_BGR2GRAY))
        
    # Aplicar subtração de fundo KNN
    fgmask = fgbg.apply(frame_equalized)

    # Remover sombras (valores intermediários)
    fgmask[fgmask == 127] = 0

    # Limpeza de ruído (leve e rápida)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel_open, iterations=1)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel_close, iterations=2)

    # Detectar contornos dos objetos claramente definidos
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Criar máscara vazia para contornos precisos
    precise_mask = np.zeros_like(fgmask)

    for cnt in contours:
        if cv2.contourArea(cnt) >= area:  # Ignorar pequenos ruídos
            cv2.drawContours(precise_mask, [cnt], -1, 255, -1)  # Preenche o objeto detectado

    # Extrair objetos em movimento (sem borrões)
    # moving_objects = cv2.bitwise_and(frame, frame, mask=precise_mask)

    # Opcional: Desenhar caixa delimitadora ao redor dos objetos (facilita identificação)
    for cnt in contours:
        if cv2.contourArea(cnt) >= area:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Salvar resultado com objetos definidos
    cv2.imwrite(os.path.join(save, f"results_{idx:04}.jpg"), precise_mask)

    idx += 1

cap.release()

# Criar vídeo final
image_folder = save
output_video_path = "motion_knn.mp4"
frame_rate = get_framerate(video_path)
create_video_from_images(image_folder, output_video_path, frame_rate)

# Abrir vídeo automaticamente após criação
os.system(f"start {output_video_path}")

# V4 - Motrackers

In [55]:
!pip install motrackers

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------- ----------------- 1.3/2.3 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 8.3 MB/s eta 0:00:00


In [45]:
os.system(f"start {output_video_path}")


0